# QIIME2 analysis of qiita ID 10249 
### Antibiotics, birth mode, and diet shape microbiome maturation during early life (ECAM)


#### Abstract

Early childhood is a critical stage for the foundation and development of both the microbiome and host. Early-life antibiotic exposures, cesarean section, and formula feeding could disrupt microbiome establishment and adversely affect health later in life. We profiled microbial development during the first two years of life in a cohort of 43 US infants, and identify multiple disturbances associated with antibiotic exposures, cesarean section, and diet. Antibiotics delayed microbiome development and suppressed Clostridiales, including Lachnospiraceae. Cesarean section led to depleted Bacteroidetes populations, altering establishment of maternal bacteria. Formula-feeding was associated with age-dependent diversity deviations. These findings illustrate the complexity of early-life microbiome development, and microbiota disturbances with antibiotic use, cesarean section, and formula feeding that may contribute to obesity, asthma, and other disorders.


### 1.1 “Parameters” that specify the input data locations

In [1]:
# Input data locations
%env INPUT_BIOM_TABLE_PATH=../../data/ECAM-Qiita-10249/table-matched.biom
%env INPUT_REPRESENTATIVE_SEQUENCES_PATH=../../data/ECAM-Qiita-10249/rep-seqs.fa
%env INPUT_SAMPLE_METADATA_PATH=../../data/ECAM-Qiita-10249/metadata-matched.tsv
# Rarefaction depth used in the "Diversity Analyses" section
%env RAREFACTION_DEPTH=2000
# classifier from (qiime2 downloaded 5/31/2019)
%env FEATURE_CLASSIFIER_QZA_PATH=../../data/data-assests/gg-13-8-99-515-806-nb-classifier.qza
# Output directory
%env OUTPUT_DIRECTORY=../../data/ECAM-Qiita-10249/q2-analysis

env: INPUT_BIOM_TABLE_PATH=../../data/ECAM-Qiita-10249/table-matched.biom
env: INPUT_REPRESENTATIVE_SEQUENCES_PATH=../../data/ECAM-Qiita-10249/rep-seqs.fa
env: INPUT_SAMPLE_METADATA_PATH=../../data/ECAM-Qiita-10249/metadata-matched.tsv
env: RAREFACTION_DEPTH=2000
env: FEATURE_CLASSIFIER_QZA_PATH=../../data/data-assests/gg-13-8-99-515-806-nb-classifier.qza
env: OUTPUT_DIRECTORY=../../data/ECAM-Qiita-10249/q2-analysis


### 1.2 Setting up output and environment 

In [3]:
!source activate qiime2-2019.7

/bin/sh: activate: No such file or directory


### 1.3 Importing Data

In [4]:
!qiime tools import \
    --type "FeatureTable[Frequency]" \
    --input-path $INPUT_BIOM_TABLE_PATH \
    --output-path $OUTPUT_DIRECTORY/table.qza

Imported ../../data/ECAM-Qiita-10249/table-matched.biom as BIOMV210DirFmt to ../../data/ECAM-Qiita-10249/q2-analysis/table.qza


In [5]:
!qiime tools import \
    --type "FeatureData[Sequence]" \
    --input-path $INPUT_REPRESENTATIVE_SEQUENCES_PATH \
    --output-path $OUTPUT_DIRECTORY/rep-seqs.qza

Imported ../../data/ECAM-Qiita-10249/rep-seqs.fa as DNASequencesDirectoryFormat to ../../data/ECAM-Qiita-10249/q2-analysis/rep-seqs.qza


### 1.4 Summarize the (filtered) table and representative sequences

In [8]:
!qiime feature-table summarize \
    --i-table $OUTPUT_DIRECTORY/table.qza \
    --o-visualization $OUTPUT_DIRECTORY/table.qzv \
    --m-sample-metadata-file $INPUT_SAMPLE_METADATA_PATH

Saved Visualization to: ../../data/ECAM-Qiita-10249/q2-analysis/table.qzv


In [9]:
!qiime feature-table tabulate-seqs \
    --i-data $OUTPUT_DIRECTORY/rep-seqs.qza \
    --o-visualization $OUTPUT_DIRECTORY/rep-seqs.qzv

Saved Visualization to: ../../data/ECAM-Qiita-10249/q2-analysis/rep-seqs.qzv


### 1.5 Generate phylogenetic trees

In [10]:
!qiime fragment-insertion sepp \
    --i-representative-sequences $OUTPUT_DIRECTORY/rep-seqs.qza \
    --o-tree $OUTPUT_DIRECTORY/insertion-tree.qza \
    --o-placements $OUTPUT_DIRECTORY/tree-placements.qza


Saved Phylogeny[Rooted] to: ../../data/ECAM-Qiita-10249/q2-analysis/insertion-tree.qza
Saved Placements to: ../../data/ECAM-Qiita-10249/q2-analysis/tree-placements.qza


### 1.6 Diversity Analyses (all core-metrics & beta-diversity aitchison)

In [17]:
!qiime diversity core-metrics-phylogenetic \
    --i-phylogeny $OUTPUT_DIRECTORY/insertion-tree.qza \
    --i-table $OUTPUT_DIRECTORY/table.qza \
    --p-sampling-depth $RAREFACTION_DEPTH \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --output-dir $OUTPUT_DIRECTORY/core-metrics-results

Saved FeatureTable[Frequency] to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metri

In [18]:
!qiime diversity beta \
    --i-table $OUTPUT_DIRECTORY/table.qza \
    --p-metric aitchison \
    --p-pseudocount 1 \
    --o-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/aitchison_distance_matrix.qza 

!qiime diversity pcoa \
    --i-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/aitchison_distance_matrix.qza \
    --o-pcoa $OUTPUT_DIRECTORY/core-metrics-results/aitchison_pcoa_results.qza

!qiime emperor plot \
    --i-pcoa $OUTPUT_DIRECTORY/core-metrics-results/aitchison_pcoa_results.qza \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --o-visualization $OUTPUT_DIRECTORY/core-metrics-results/aitchisons_emperor.qzv

Saved DistanceMatrix to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/aitchison_distance_matrix.qza
Saved PCoAResults to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/aitchison_pcoa_results.qza
Saved Visualization to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/aitchisons_emperor.qzv


In [19]:
!qiime diversity beta-phylogenetic \
    --i-table $OUTPUT_DIRECTORY/core-metrics-results/rarefied_table.qza \
    --p-metric generalized_unifrac \
    --i-phylogeny $OUTPUT_DIRECTORY/insertion-tree.qza \
    --p-alpha 1 \
    --o-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha1_distance_matrix.qza 

!qiime diversity pcoa \
    --i-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha1_distance_matrix.qza \
    --o-pcoa $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha1_pcoa_results.qza

!qiime emperor plot \
    --i-pcoa $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha1_pcoa_results.qza \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --o-visualization $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha1_emperor.qzv

!qiime diversity beta-phylogenetic \
    --i-table $OUTPUT_DIRECTORY/core-metrics-results/rarefied_table.qza \
    --p-metric generalized_unifrac \
    --i-phylogeny $OUTPUT_DIRECTORY/insertion-tree.qza \
    --p-alpha 0.5 \
    --o-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha5_distance_matrix.qza 

!qiime diversity pcoa \
    --i-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha5_distance_matrix.qza \
    --o-pcoa $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha5_pcoa_results.qza

!qiime emperor plot \
    --i-pcoa $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha5_pcoa_results.qza \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --o-visualization $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha5_emperor.qzv

!qiime diversity beta-phylogenetic \
    --i-table $OUTPUT_DIRECTORY/core-metrics-results/rarefied_table.qza \
    --p-metric generalized_unifrac \
    --i-phylogeny $OUTPUT_DIRECTORY/insertion-tree.qza \
    --p-alpha 0 \
    --o-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha0_distance_matrix.qza 

!qiime diversity pcoa \
    --i-distance-matrix $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha0_distance_matrix.qza \
    --o-pcoa $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha0_pcoa_results.qza

!qiime emperor plot \
    --i-pcoa $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha0_pcoa_results.qza \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --o-visualization $OUTPUT_DIRECTORY/core-metrics-results/gUniFrac_alpha0_emperor.qzv


Saved DistanceMatrix % Properties('phylogenetic') to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha1_distance_matrix.qza
Saved PCoAResults to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha1_pcoa_results.qza
Saved Visualization to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha1_emperor.qzv
Saved DistanceMatrix % Properties('phylogenetic') to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha5_distance_matrix.qza
Saved PCoAResults to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha5_pcoa_results.qza
Saved Visualization to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha5_emperor.qzv
Saved DistanceMatrix % Properties('phylogenetic') to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFrac_alpha0_distance_matrix.qza
Saved PCoAResults to: ../../data/ECAM-Qiita-10249/q2-analysis/core-metrics-results/gUniFr

### 1.7 Taxonomic Analysis

In [23]:
!qiime feature-classifier classify-sklearn \
    --i-classifier $FEATURE_CLASSIFIER_QZA_PATH \
    --i-reads $OUTPUT_DIRECTORY/rep-seqs.qza \
    --o-classification $OUTPUT_DIRECTORY/taxonomy.qza

Saved FeatureData[Taxonomy] to: ../../data/ECAM-Qiita-10249/q2-analysis/taxonomy.qza


### 1.7.2 Summarize the Taxonomic Classifications

In [24]:
!qiime metadata tabulate \
    --m-input-file $OUTPUT_DIRECTORY/taxonomy.qza \
    --o-visualization $OUTPUT_DIRECTORY/taxonomy.qzv

Saved Visualization to: ../../data/ECAM-Qiita-10249/q2-analysis/taxonomy.qzv


### 1.8.1 CTF 

In [2]:
!qiime gemelli ctf --i-table $OUTPUT_DIRECTORY/table.qza \
                   --m-sample-metadata-file $INPUT_SAMPLE_METADATA_PATH \
                   --p-individual-id-column host_subject_id \
                   --p-state-column month \
                   --m-feature-metadata-file $OUTPUT_DIRECTORY/taxonomy.qza \
                   --p-min-sample-count 0\
                   --p-min-feature-count 5\
                   --p-max-iterations-rptm 5 \
                   --p-n-initializations 5 \
                   --p-max-iterations-als 5 \
                   --output-dir $OUTPUT_DIRECTORY/ctf-results \
                   --p-n-components 2 \
                   --verbose

/Users/cmartino/Dropbox/bin/gemelli/gemelli/preprocessing.py:356: RuntimeWarning: Subject(s) (38.0,4.0,5.0,10.0,22.0,21.0,34.0,56.0,18.0,52.0,25.0,44.0,46.0,41.0,27.0,2.0,45.0,33.0,7.0,37.0,57.0,30.0,8.0,35.0,11.0,17.0,24.0,20.0,31.0,32.0,1.0,49.0,14.0,43.0,53.0,36.0,12.0,55.0,42.0,16.0) contains multiple samples. Multiple subject counts will be meaned across samples by subject.
  RuntimeWarning)
Saved PCoAResults % Properties('biplot') to: ../../data/ECAM-Qiita-10249/q2-analysis/ctf-results/subject_biplot.qza
Saved PCoAResults % Properties('biplot') to: ../../data/ECAM-Qiita-10249/q2-analysis/ctf-results/state_biplot.qza
Saved DistanceMatrix to: ../../data/ECAM-Qiita-10249/q2-analysis/ctf-results/distance_matrix.qza
Saved SampleData[SampleTrajectory] to: ../../data/ECAM-Qiita-10249/q2-analysis/ctf-results/state_subject_ordination.qza
Saved FeatureData[FeatureTrajectory] to: ../../data/ECAM-Qiita-10249/q2-analysis/ctf-results/state_feature_ordination.qza
